In [53]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

In [54]:
column_names = list(range(0,10))
column_names.append('Classification')

df = pd.read_csv("glass.data", names = column_names) 
df

,0,1,2,3,4,5,6,7,8,9,Classification
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
209,210,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7
210,211,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7
211,212,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7
212,213,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7


In [55]:
df = df.drop(columns = [0], axis = 1)
df

,1,2,3,4,5,6,7,8,9,Classification
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7


In [56]:
df.dtypes

1                 float64
2                 float64
3                 float64
4                 float64
5                 float64
6                 float64
7                 float64
8                 float64
9                 float64
Classification      int64
dtype: object

In [57]:
#find number of null entries to see if there is a need to clean the dataset
df.isnull().sum()
#df = df.dropna()

1                 0
2                 0
3                 0
4                 0
5                 0
6                 0
7                 0
8                 0
9                 0
Classification    0
dtype: int64

In [58]:
df.nunique()

1                 178
2                 142
3                  94
4                 118
5                 133
6                  65
7                 143
8                  34
9                  32
Classification      6
dtype: int64

In [59]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
df['Classification'] = label_encoder.fit_transform(df['Classification'])
df

,1,2,3,4,5,6,7,8,9,Classification
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,0
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,0
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,0
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,0
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,0
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,5
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,5
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,5
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,5


In [60]:
X = df.drop('Classification', axis=1)
y = df['Classification']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [61]:
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

tf.random.set_seed(42)
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(128, activation='relu', input_shape = (9,)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(6, activation='softmax'))

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy",
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

from sklearn.model_selection import KFold
kf = KFold(n_splits=10, random_state =42, shuffle=True)

sum_acc_score = 0
sum_prec_score = 0
sum_recall_score = 0
sum_f1_score = 0


for train_index, test_index in kf.split(X):
    X_train = pd.DataFrame()
    y_train = pd.DataFrame()

    X_test = pd.DataFrame()
    y_test = pd.DataFrame()
    
    for index in train_index:
        X_train = X_train.append(X.iloc[index])
        y_train = y_train.append(pd.Series(y.iloc[index]), ignore_index = True)
        
    for index in test_index:
        X_test = X_test.append(X.iloc[index])
        y_test = y_test.append(pd.Series(y.iloc[index]), ignore_index = True)

    train_X = tf.convert_to_tensor(X_train)
    test_X = tf.convert_to_tensor(X_test)
    
    
    #one hot encode y_train and y_test
    train_y = tf.keras.utils.to_categorical(y_train, num_classes=6)
    test_y = tf.keras.utils.to_categorical(y_test, num_classes=6)

    history = model.fit(train_X, train_y, epochs=50, validation_data = (test_X, test_y),callbacks=[callback])
    predictions = model.predict(test_X)
    
    predicted_classes = predictions.argmax(axis=1)
    test_y_classes = test_y.argmax(axis=1)
    
    sum_acc_score += accuracy_score(test_y_classes, predicted_classes)
    sum_prec_score += precision_score(test_y_classes, predicted_classes, average = 'micro')
    sum_recall_score += recall_score(test_y_classes, predicted_classes, average = 'micro')
    sum_f1_score += f1_score(test_y_classes, predicted_classes, average = 'micro')

Epoch 1/50
6/6 [==============================] - 4s 260ms/step - loss: 3.7902 - accuracy: 0.1979 - precision: 0.2308 - recall: 0.1875 - val_loss: 3.4497 - val_accuracy: 0.2273 - val_precision: 0.2273 - val_recall: 0.2273
Epoch 2/50
6/6 [==============================] - 0s 12ms/step - loss: 2.6592 - accuracy: 0.3594 - precision: 0.3594 - recall: 0.3594 - val_loss: 1.6355 - val_accuracy: 0.2273 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/50
6/6 [==============================] - 0s 10ms/step - loss: 2.1798 - accuracy: 0.3177 - precision: 0.1471 - recall: 0.0260 - val_loss: 1.7743 - val_accuracy: 0.2273 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/50
6/6 [==============================] - 0s 19ms/step - loss: 1.8260 - accuracy: 0.3646 - precision: 0.3663 - recall: 0.1927 - val_loss: 2.5848 - val_accuracy: 0.2273 - val_precision: 0.3333 - val_recall: 0.0455
Epoch 5/50
6/6 [==============================] - 0s 11ms/step - loss: 1.6020 - accuracy: 0.3750 - 

6/6 [==============================] - 0s 12ms/step - loss: 1.1088 - accuracy: 0.4688 - precision: 0.5280 - recall: 0.3438 - val_loss: 1.0365 - val_accuracy: 0.5455 - val_precision: 0.6111 - val_recall: 0.5000
Epoch 38/50
6/6 [==============================] - 0s 14ms/step - loss: 1.1380 - accuracy: 0.4948 - precision: 0.5224 - recall: 0.3646 - val_loss: 1.0556 - val_accuracy: 0.6364 - val_precision: 0.8333 - val_recall: 0.2273
Epoch 39/50
6/6 [==============================] - 0s 12ms/step - loss: 1.0698 - accuracy: 0.5156 - precision: 0.6203 - recall: 0.2552 - val_loss: 1.0328 - val_accuracy: 0.4545 - val_precision: 0.8000 - val_recall: 0.3636
Epoch 40/50
6/6 [==============================] - 0s 12ms/step - loss: 1.0560 - accuracy: 0.5052 - precision: 0.6341 - recall: 0.2708 - val_loss: 1.0504 - val_accuracy: 0.5909 - val_precision: 0.7000 - val_recall: 0.3182
Epoch 41/50
6/6 [==============================] - 0s 12ms/step - loss: 1.0737 - accuracy: 0.5156 - precision: 0.6087 - reca

Epoch 24/50
6/6 [==============================] - 0s 18ms/step - loss: 1.0400 - accuracy: 0.5312 - precision: 0.6364 - recall: 0.3646 - val_loss: 0.8649 - val_accuracy: 0.6364 - val_precision: 0.6500 - val_recall: 0.5909
Epoch 25/50
6/6 [==============================] - 0s 17ms/step - loss: 0.9618 - accuracy: 0.5677 - precision: 0.6304 - recall: 0.4531 - val_loss: 0.9450 - val_accuracy: 0.6364 - val_precision: 0.5882 - val_recall: 0.4545
Epoch 26/50
6/6 [==============================] - 0s 19ms/step - loss: 1.0271 - accuracy: 0.5312 - precision: 0.5669 - recall: 0.3750 - val_loss: 0.8110 - val_accuracy: 0.6364 - val_precision: 0.7647 - val_recall: 0.5909
Epoch 27/50
6/6 [==============================] - 0s 21ms/step - loss: 0.9809 - accuracy: 0.5260 - precision: 0.6195 - recall: 0.3646 - val_loss: 0.8846 - val_accuracy: 0.5909 - val_precision: 0.7333 - val_recall: 0.5000
Epoch 28/50
6/6 [==============================] - 0s 19ms/step - loss: 0.9709 - accuracy: 0.5417 - precision: 0

Epoch 17/50
6/6 [==============================] - 0s 10ms/step - loss: 1.0487 - accuracy: 0.5156 - precision: 0.5280 - recall: 0.4427 - val_loss: 1.0071 - val_accuracy: 0.5909 - val_precision: 0.5882 - val_recall: 0.4545
Epoch 18/50
6/6 [==============================] - 0s 10ms/step - loss: 0.9478 - accuracy: 0.5521 - precision: 0.6014 - recall: 0.4479 - val_loss: 0.8375 - val_accuracy: 0.5000 - val_precision: 0.5625 - val_recall: 0.4091
Epoch 19/50
1/1 [==============================] - 0s 30ms/step
Epoch 1/50
6/6 [==============================] - 0s 28ms/step - loss: 0.9158 - accuracy: 0.5677 - precision: 0.6099 - recall: 0.4479 - val_loss: 0.9187 - val_accuracy: 0.5455 - val_precision: 0.5789 - val_recall: 0.5000
Epoch 2/50
6/6 [==============================] - 0s 11ms/step - loss: 0.9234 - accuracy: 0.6042 - precision: 0.6127 - recall: 0.5521 - val_loss: 0.8430 - val_accuracy: 0.4545 - val_precision: 0.6000 - val_recall: 0.2727
Epoch 3/50
6/6 [==============================] - 

7/7 [==============================] - 0s 10ms/step - loss: 1.0083 - accuracy: 0.5751 - precision: 0.5697 - recall: 0.4870 - val_loss: 1.3357 - val_accuracy: 0.4762 - val_precision: 0.4737 - val_recall: 0.4286
Epoch 12/50
7/7 [==============================] - 0s 8ms/step - loss: 1.2219 - accuracy: 0.5285 - precision: 0.5385 - recall: 0.5078 - val_loss: 1.1062 - val_accuracy: 0.3810 - val_precision: 0.5000 - val_recall: 0.3333
Epoch 13/50
7/7 [==============================] - 0s 9ms/step - loss: 1.1756 - accuracy: 0.4249 - precision: 0.4538 - recall: 0.3057 - val_loss: 1.6604 - val_accuracy: 0.3810 - val_precision: 0.4000 - val_recall: 0.3810
Epoch 14/50
7/7 [==============================] - 0s 9ms/step - loss: 0.9809 - accuracy: 0.6218 - precision: 0.6364 - recall: 0.5803 - val_loss: 2.2513 - val_accuracy: 0.5238 - val_precision: 0.5238 - val_recall: 0.5238
Epoch 15/50
7/7 [==============================] - 0s 10ms/step - loss: 1.0718 - accuracy: 0.5959 - precision: 0.5965 - recall:

Epoch 9/50
7/7 [==============================] - 0s 10ms/step - loss: 0.9135 - accuracy: 0.5751 - precision: 0.6104 - recall: 0.4870 - val_loss: 0.7698 - val_accuracy: 0.5714 - val_precision: 0.6316 - val_recall: 0.5714
Epoch 10/50
7/7 [==============================] - 0s 10ms/step - loss: 0.9861 - accuracy: 0.5699 - precision: 0.5771 - recall: 0.5233 - val_loss: 0.6496 - val_accuracy: 0.7143 - val_precision: 0.7143 - val_recall: 0.7143
Epoch 11/50
7/7 [==============================] - 0s 10ms/step - loss: 1.0652 - accuracy: 0.5389 - precision: 0.5455 - recall: 0.4974 - val_loss: 0.6473 - val_accuracy: 0.7619 - val_precision: 0.9167 - val_recall: 0.5238
Epoch 12/50
1/1 [==============================] - 0s 42ms/step
Epoch 1/50
7/7 [==============================] - 0s 22ms/step - loss: 1.0658 - accuracy: 0.4819 - precision: 0.6848 - recall: 0.3264 - val_loss: 1.0998 - val_accuracy: 0.6667 - val_precision: 0.6842 - val_recall: 0.6190
Epoch 2/50
7/7 [==============================] - 

In [62]:
print('Average Accuracy Score: ', sum_acc_score/10)
print('Average Precision Score: ', sum_prec_score/10)
print('Average Recall Score: ', sum_recall_score/10)
print('Average F1-Score: ', sum_f1_score/10)



Average Accuracy Score:  0.5608225108225109
Average Precision Score:  0.5608225108225109
Average Recall Score:  0.5608225108225109
Average F1-Score:  0.5608225108225109
